# Problem 1

## (1) Marginal-cost pricing toll patteren

The link performance function:

$t_{12} = 60 + x_{12}$

$t_{13} = 10 + 10x_{13}$

$t_{24} = 10 + 10x_{24}$

$t_{34} = 60 + x_{34}$

$t_{32} = 10 + x_{32}$

Origin: Node 1; Destination: Node 4

Total Traffic Flow: 6 Vehicles

Possible Paths:

1. Node 1 $\rightarrow$ Node 2 $\rightarrow$ Node 4

2. Node 1 $\rightarrow$ Node 3 $\rightarrow$ Node 4

3. Node 1 $\rightarrow$ Node 3 $\rightarrow$ Node 2 $\rightarrow$ Node 4

The goal of System Optimum (SO) is to minimize the total travel time in the network, so we can derive the following formulation:

$Z = min\space\Sigma_{i,j}\space x_{ij}t_{ij} = min \space x_{12}(60+x_{12})+x_{13}(10+10x_{13})+x_{24}(10+10x_{24})+x_{32}(10+x_{32})+x_{34}(60+x_{34})$

The constraints are the traffic flow conservation on all nodes and postive traffice flow.

$s.t.$

$x_{12}+x_{13}-6 = 0$

$x_{24}-x_{12}-x_{32} = 0$

$x_{32}+x_{34}-x_{13} = 0$

$6-x_{24}-x_{34} = 0$

$x_{ij} \geq 0 \space \forall i,j$

In [1]:
import numpy as np
from scipy import optimize
from gurobipy import *

In [2]:
# create an optimization model called Marginal_Travel_Time
model_1_1 = Model('Marginal_Travel_Time')

# add variables into this model
x = {}
x[1,2] = model_1_1.addVar(vtype=GRB.CONTINUOUS)
x[1,3] = model_1_1.addVar(vtype=GRB.CONTINUOUS)
x[2,4] = model_1_1.addVar(vtype=GRB.CONTINUOUS)
x[3,2] = model_1_1.addVar(vtype=GRB.CONTINUOUS)
x[3,4] = model_1_1.addVar(vtype=GRB.CONTINUOUS)

# set the objective function in this model
obj = x[1,2]*(60+x[1,2]) + x[1,3]*(10+10*x[1,3]) + x[2,4]*(10+10*x[2,4]) + x[3,2]*(10+x[3,2]) + x[3,4]*(60+x[3,4])
model_1_1.setObjective(obj)

# set the constraints in this model
model_1_1.addConstr(x[1,2]+x[1,3]-6==0)
model_1_1.addConstr(x[2,4]-x[1,2]-x[3,2]==0)
model_1_1.addConstr(x[3,2]+x[3,4]-x[1,3]==0)
model_1_1.addConstr(6-x[2,4]-x[3,4]==0)

model_1_1.optimize()
model_1_1.write('Marginal_Travel_Time.lp')

Restricted license - for non-production use only - expires 2023-10-25
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 4 rows, 5 columns and 10 nonzeros
Model fingerprint: 0x71bbae03
Model has 5 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 6e+01]
  QObjective range [2e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+00, 6e+00]
Presolve removed 1 rows and 0 columns
Presolve time: 0.00s
Presolved: 3 rows, 5 columns, 7 nonzeros
Presolved model has 5 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 2.000e+00
 Factor NZ  : 6.000e+00
 Factor Ops : 1.400e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.13985542e+06 -1.071745

In [3]:
sol_list = []

print('Traffic flow from Node 1 to Node 2 is: {:.2f}'.format(x[1,2].X))
sol_list.append(x[1,2].X)
print('Traffic flow from Node 1 to Node 3 is: {:.2f}'.format(x[1,3].X))
sol_list.append(x[1,3].X)
print('Traffic flow from Node 2 to Node 4 is: {:.2f}'.format(x[2,4].X))
sol_list.append(x[2,4].X)
print('Traffic flow from Node 3 to Node 2 is: {:.2f}'.format(x[3,2].X))
sol_list.append(x[3,2].X)
print('Traffic flow from Node 3 to Node 4 is: {:.2f}'.format(x[3,4].X))
sol_list.append(x[3,4].X)

sol_list = np.asarray(sol_list)

Traffic flow from Node 1 to Node 2 is: 3.00
Traffic flow from Node 1 to Node 3 is: 3.00
Traffic flow from Node 2 to Node 4 is: 3.00
Traffic flow from Node 3 to Node 2 is: 0.00
Traffic flow from Node 3 to Node 4 is: 3.00


In [4]:
dev_list = np.array([1,10,10,1,1])
marginal_cost = sol_list*dev_list

print('Marginal cost from Node 1 to Node 2 is: {:.2f}'.format(marginal_cost[0]))
print('Marginal cost from Node 1 to Node 3 is: {:.2f}'.format(marginal_cost[1]))
print('Marginal cost from Node 2 to Node 4 is: {:.2f}'.format(marginal_cost[2]))
print('Marginal cost from Node 3 to Node 2 is: {:.2f}'.format(marginal_cost[3]))
print('Marginal cost from Node 3 to Node 4 is: {:.2f}'.format(marginal_cost[4]))

Marginal cost from Node 1 to Node 2 is: 3.00
Marginal cost from Node 1 to Node 3 is: 30.00
Marginal cost from Node 2 to Node 4 is: 30.00
Marginal cost from Node 3 to Node 2 is: 0.00
Marginal cost from Node 3 to Node 4 is: 3.00


## (2) UE to SO

According to Hearn and Ramana, 1998, the model that can convert User Equilibrium (UE) to System Optimum (SO) should be formulated as the following:

$Z = min \space \Sigma_{i,j} \space x_{ij}^{SO} \tau_{ij}$

$s.t.$

$\Sigma_{a} \space x_{a}^{SO}(t_{a}(x_{a}^{SO}+\tau_{a})) = \Sigma_{rs} \space q_{rs}\lambda^{rs}, \space \space \forall r,s$

$\Sigma_{a} \space \delta_{ak}^{rs}(t_{a}(x_{a}^{SO}+\tau_{a})) \geq \lambda_{rs}, \space\space \forall k,r,s$

$\tau_a \geq 0, \space\space \forall a$

Therefore, we can formulate the model numerically as the following format:

$min \space Z = 3(\tau_{12}+\tau_{13}+\tau_{24}+\tau_{34})$

$s.t.$

$618 + 3(\tau_{12}+\tau_{13}+\tau_{24}+\tau_{34}) - 6\lambda = 0$

$103 + \tau_{12} + \tau_{24} - \lambda \geq 0$

$103 + \tau_{13} + \tau_{34} - \lambda \geq 0$

$90 + \tau_{13} + \tau_{24} + \tau_{32} - \lambda \geq 0$

$\tau_{12}, \tau_{13}, \tau_{24}, \tau_{32}, \tau_{34} \geq 0$

In [5]:
# create an optimization model called UE_to_SO
model_1_2 = Model('UE_to_SO')

# add variables into this model
tau = {}
tau[1,2] = model_1_2.addVar(vtype=GRB.CONTINUOUS)
tau[1,3] = model_1_2.addVar(vtype=GRB.CONTINUOUS)
tau[2,4] = model_1_2.addVar(vtype=GRB.CONTINUOUS)
tau[3,2] = model_1_2.addVar(vtype=GRB.CONTINUOUS)
tau[3,4] = model_1_2.addVar(vtype=GRB.CONTINUOUS)
lamb = model_1_2.addVar(vtype=GRB.CONTINUOUS)

# set the objective function in this model
obj = 3*(tau[1,2]+tau[1,3]+tau[2,4]+tau[3,4])
model_1_2.setObjective(obj)

# set the constraints in this model
model_1_2.addConstr(618+3*(tau[1,2]+tau[1,3]+tau[2,4]+tau[3,4])-6*lamb==0)
model_1_2.addConstr(103+tau[1,2]+tau[2,4]-lamb >= 0)
model_1_2.addConstr(103+tau[1,3]+tau[3,4] >= 0)
model_1_2.addConstr(90+tau[1,3]+tau[2,4]+tau[3,2]-lamb >= 0)

model_1_2.optimize()
model_1_2.write('UE_to_SO.lp')

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 4 rows, 6 columns and 14 nonzeros
Model fingerprint: 0x07558e9a
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [3e+00, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [9e+01, 6e+02]
Presolve removed 4 rows and 6 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  0.000000000e+00


In [6]:
print('Toll on the link from Node 1 to Node 2 is: {:.2f}'.format(tau[1,2].X))
print('Toll on the link from Node 1 to Node 3 is: {:.2f}'.format(tau[1,3].X))
print('Toll on the link from Node 2 to Node 4 is: {:.2f}'.format(tau[2,4].X))
print('Toll on the link from Node 3 to Node 2 is: {:.2f}'.format(tau[3,2].X))
print('Toll on the link from Node 3 to Node 4 is: {:.2f}'.format(tau[3,4].X))
print('The balanced travel time on the OD pair is: {:.2f}'.format(lamb.X))

Toll on the link from Node 1 to Node 2 is: 0.00
Toll on the link from Node 1 to Node 3 is: 0.00
Toll on the link from Node 2 to Node 4 is: 0.00
Toll on the link from Node 3 to Node 2 is: 13.00
Toll on the link from Node 3 to Node 4 is: 0.00
The balanced travel time on the OD pair is: 103.00


# Problem 2

$t_1(x_1) = 2+0.01x_1^4$

$t_2(x_2) = 1+0.02x_2^4$

$\frac{x_1}{x_1+x_2} = \frac{1}{1+e^(\theta(t_1-t_2))}, \space\space \theta=0.5$

$x_1+x_2=6$

## (1) Formulate the stochastic UE minimization problem

Based on Fisk's logit-based formulation, the formulation should be:

$min \space Z = \Sigma_{a \in A} \int_{0}^{x_a} t(\tilde{\omega})\, d\tilde{\omega} + \frac{1}{\theta} \Sigma_{rs} \Sigma_{k} \space f_k^{rs}lnf_k^{rs}$

$s.t.$

$\Sigma_{k} f_k^{rs} = q_{rs}, \space\space \forall r,s$

$f_k^{rs} \geq 0, \space\space \forall k,r,s$

$x_a = \Sigma_{rs} \Sigma_{k} f_k^{rs}\delta_{a,k}^{rs}, \space a \in A$

Therefore, for our question, we can formulate the optimization model as the following:

$min \space Z = 2x_1 + \frac{x_1^5}{500} + x_2 + \frac{x_2^5}{250} + 2(x_1lnx_1 + x_2lnx_2)$

$s.t.$

$x_1 + x_2 = 6$

$x_1, x_2 \geq 0$

## (2) Successive Average Method

The structure of successive average method

0. Initialization. Perform a stochastic network loading based on $t_a = t_a(0), \space \forall a$. This yields $x^1$. Set counter $n:=1$.

1. Update. Set $t_a^n = t_a(x_a^n), \space \forall a$.

2. Direct finding. Perform a stochastic network loading based on $t_a^n, \space \forall a$. This yields flows $y^n$.

3. Move. Set $x^{n+1} = x^n +\frac{1}{n}(y^n-x^n)$

4. Convergence test. If convergence is attained, stop; otherwise, $n:=n+1$, and to to Step 1.

In [7]:
def travel_time(x):
    x1, x2 = x[0], x[1]
    t1 = 2+0.01*x1**4
    t2 = 1+0.02*x2**4
    return np.array([t1,t2])

def mode_choice(t):
    t1, t2 = t[0], t[1]
    mode_1 = 6/(1+np.exp(0.5*(t1-t2)))
    mode_2 = 6 - mode_1
    return np.array([mode_1, mode_2])

def initialization():
    time = travel_time(np.zeros(2))
    flow = mode_choice(time)
    return time, flow

def convergence(new_flow, old_flow):
    epsilon = 0.001
    flow_diff = (new_flow - old_flow)**2
    result = np.sqrt(np.sum(flow_diff))/np.sum(old_flow)
    if result <= epsilon:
        print('The result is converged!')
        return True
    else:
        print('The result is not converged!')
        print('Keep going!\n')
        return False

def update(old_flow, counter):
    print('Iteration ', counter)
    print('Original Traffic Flow: ', old_flow)
    updated_travel_time = travel_time(old_flow) # return new travel time
    print('Updated Travel Time: ', updated_travel_time)
    direct_finding = mode_choice(updated_travel_time)
    print('Direct Finding: ', direct_finding)
    updated_flow = old_flow + 1/counter*(direct_finding-old_flow)
    print('Updated Flow: ', updated_flow)
    return updated_flow

In [8]:
# initialization
iteration = True
counter = 1
old_flow = initialization()[1]

# update
while(iteration):
    new_flow = update(old_flow, counter)
    convergeOrNot = convergence(new_flow, old_flow)
    if convergeOrNot==False:
        old_flow = new_flow
        counter += 1
    else:
        iteration = False
        print('\n')
        print('After {:.0f} iterations, we get the converged traffic flow.'.format(counter))
        print('The final traffic flow on both links is {:.2f} and {:.2f}. '.format(new_flow[0],new_flow[1]))

Iteration  1
Original Traffic Flow:  [2.26524401 3.73475599]
Updated Travel Time:  [2.26330552 4.89115853]
Direct Finding:  [4.72901676 1.27098324]
Updated Flow:  [4.72901676 1.27098324]
The result is not converged!
Keep going!

Iteration  2
Original Traffic Flow:  [4.72901676 1.27098324]
Updated Travel Time:  [7.00130583 1.05219024]
Direct Finding:  [0.29153155 5.70846845]
Updated Flow:  [2.51027416 3.48972584]
The result is not converged!
Keep going!

Iteration  3
Original Traffic Flow:  [2.51027416 3.48972584]
Updated Travel Time:  [2.39708604 3.96616451]
Direct Finding:  [4.11994558 1.88005442]
Updated Flow:  [3.0468313 2.9531687]
The result is not converged!
Keep going!

Iteration  4
Original Traffic Flow:  [3.0468313 2.9531687]
Updated Travel Time:  [2.86177449 2.52118846]
Direct Finding:  [2.745176 3.254824]
Updated Flow:  [2.97141748 3.02858252]
The result is not converged!
Keep going!

Iteration  5
Original Traffic Flow:  [2.97141748 3.02858252]
Updated Travel Time:  [2.779569

## (3) Analytical Solution

In this problem, we have two variables $(x_1 and x_2)$ and following two equations.

$x_1+x_2=6$

$\frac{x_1}{x_1+x_2} = \frac{1}{1+e^{0.5(1+0.01x_1^4-0.02x_2^4)}}$

Therefore, we can reeformulate our problem as the following format and solve it.

$x_1 - \frac{6}{1+e^{0.5[1+0.01x_1^4-0.02(6-x_1)^4]}} = 0$

In [9]:
power = lambda x: 0.5*(1+0.01*x**4-0.02*(6-x)**4)
equa = lambda x: x - 6/(1+np.exp(power(x)))

sol = optimize.root_scalar(equa, bracket=[2.5,3.5])
sol = np.array([sol.root, 6-sol.root])
diff_percentage = (new_flow - sol)/sol*100

In [10]:
print('The iteratively final traffic flow on both links: ', new_flow)
print('The analytically final traffic flow on both links: ', sol)
print('The error between two solutions on both links is respectively {:.3f}% and {:.3f}%.'.format(diff_percentage[0], diff_percentage[1]))

The iteratively final traffic flow on both links:  [2.96032597 3.03967403]
The analytically final traffic flow on both links:  [2.9586629 3.0413371]
The error between two solutions on both links is respectively 0.056% and -0.055%.


We can input two results into cost function to check which solution is more precise.

$\hat{c_1} = 2+0.01x_1^4+2ln(\frac{x_1}{6})$

$\hat{c_2} = 1+0.02x_2^4+2ln(\frac{x_2}{6})$

In [11]:
def cost(x):
    x1,x2 = x[0],x[1]
    c1 = 2+0.01*x1**4+2*np.log(x1/6)
    c2 = 1+0.02*x2**4+2*np.log(x2/6)
    return np.array([c1,c2])

numerical_cost = cost(new_flow)
analytical_cost = cost(sol)

In [12]:
print('The numercial cost of both links is: ', numerical_cost)
print('The analytical cost of both links is: ', analytical_cost)

The numercial cost of both links is:  [1.35507438 1.34739253]
The analytical cost of both links is:  [1.35222617 1.35222617]


From the above cell, we can infer that the analytical result is better than iterative one since the cost on both links matches each other better. This outcome is not surprising for me because we allow a certain error in iterative method. Although analytical method can bring a precise result to us, it may be difficult to be solved if the network becomes much more complicated. Iterative method may be more applicable when we face a large and complex network.